This notebook is designed to show some of the many different ways of leveraging the power of advanced models like OpenAI's ChatGPT through their APIs rather than simply a web interface.

#### Requirements

As this field is evolving at an extremely rapid pace (e.g. OpenAI has only recently deprecated several of their model endpoints), ensuring stability with such tools can be tricky. The following package versions below, __when run in a Colab environment__, yield consistent results.

In [ ]:
%%capture
!pip install langchain==0.3.8 \
langchain-openai==0.2.9 \
langchain-community==0.3.8 \
pypdf==5.1.0 \
chromadb==0.5.20 \
tiktoken==0.8.0 \
huggingface-hub[hf_transfer]==0.26.2 \
ctransformers[cuda]==0.2.27 \
diffusers==0.31.0 \
llama-cpp-python==0.3.2 \
openai==1.55.3 \
httpx==0.27.2 \
hf_transfer \
"protobuf<5.0.0"

This line below is added after pip install in Google Colab to force a restart, so that newly installed packages are available right away.

In [ ]:
import os
os.kill(os.getpid(), 9)

#### OpenAI API key

Additionally, you will need an OpenAI API key to run many of the below examples. You can sign up for one [here](https://openai.com/blog/openai-api). The examples presented here will cost only a few cents to run!

Please create your own key and replace it below ;)

In [ ]:
openai_api_key = ''

### Prompt engineering:

Some key points:

- Using role-playing
- Being specific in the task
- Highlighting inputs and specifying outputs

### 1. Using the OpenAI API

In [5]:
# Initialize the OpenAI API client and set your API key

import openai

openai.api_key = openai_api_key

In [6]:
# Prompt for the AI model
prompt = "Translate the following English text to French: 'Hello, how are you?'"

# Make a request to the API to generate text
# This line below starts the process of asking ChatGPT a question (a "chat completion").
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",  # Use the engine of your choice
    messages = [{"role": "user", "content": prompt}],
    max_tokens = 50
)

In [7]:
response # This is the full result returned by OpenAI when you call the ChatGPT API. It contains a lot of information, including:
# the response text, token usage, model details, and possibly multiple response "choices".

ChatCompletion(id='chatcmpl-BFmVsTdzNiddEnqfK1xn1HbEuooaa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="'Salut, comment ça va?'", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1743101104, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=22, total_tokens=31, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [9]:
response.choices[0].message.content

"'Salut, comment ça va?'"

#### System prompts

In ChatGPT's API, the system prompt is a special message that lets you instruct or guide the AI’s behavior before the conversation starts.

It sets the tone, role, personality, or boundaries for how ChatGPT should respond.

In [6]:
# Prompt for the AI model
system_prompt = "You are a sassy culinary instructor that gives sarcastic replies"
prompt = "Give instructions to cook vegetable samosas"

# Make a request to the API to generate text
response = openai.chat.completions.create(
    model="gpt-3.5-turbo",  # Use the engine of your choice
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt}],
    max_tokens = 50
)

In [7]:
response.choices[0].message.content

'Oh, you want to tackle the mighty vegetable samosas, do you? Well, start by finely dicing your veggies, making sure to channel your inner perfectionist. Then, sauté them in a pan with some spices, because bland sam'

#### Function calling

Imagine you have a python function:

```python
def get_current_weather(location, unit):
    ### A request is made to an API with a specific format
    ### returns some result
```

You want your user to write a question in natural language, and use that input to call the function to get the current weather.

In [8]:
# Example user input
user_question = "I'm interested in the weather in Bozeman. I'm old-school so I like it in F?"

This code is using GPT-4 to answer a question from a user. But here's the twist:

*If the user's question is about the weather, ChatGPT is smart enough to say:
"Hmm, I should call a special tool (a **function**) that gets live weather info."*

In [9]:
# Use GPT to interpret the user's question
# and return the function arguments
completion = openai.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": user_question}],
    functions=[
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city with its accompanying state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string",
                         "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
],
function_call="auto",
)

✅ In simple terms:

- You're teaching ChatGPT how to **use a tool** (a function) to get live weather.
- ChatGPT decides **when** to use that tool.
- It **fills in the blanks** like city and unit automatically from the user's question.

In [10]:
completion.choices[0].message.function_call.arguments

'{\n  "location": "Bozeman, MT",\n  "unit": "fahrenheit"\n}'

#### A worked-out example leveraging OpenAI and a local DataFrame

In [11]:
import pandas as pd
import json

df = pd.read_csv("data/results.csv")

df["date"] = pd.to_datetime(df["date"])
df.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1969-11-01,Italy,France,1,0,Euro,Novara,Italy,False
1,1969-11-01,Denmark,England,4,3,Euro,Aosta,Italy,True
2,1969-11-02,England,France,2,0,Euro,Turin,Italy,True
3,1969-11-02,Italy,Denmark,3,1,Euro,Turin,Italy,False
4,1970-07-06,England,West Germany,5,1,World Cup,Genova,Italy,True


Write a function to retrieve specified data: all matches in a country from the starting year to the end year:

In [12]:
def matches_finder(country: str, start_year: int, end_year: int):
    return df.loc[
        (df["country"] == country) &
        (start_year <= df["date"].dt.year) &
        (df["date"].dt.year <= end_year)
    ]

In the cell below, we describe a function that might be used to query our DataFrame. Feel free to change the `"user"` prompt in the `messages` list.

In [13]:
query = "Tell me about matches that took place in Italy between 1980 up until the end of the 20th century"

completion = openai.chat.completions.create(
    model="gpt-4-0613",
    messages=[{"role": "user", "content": query}],
    functions=[
    {
        "name": "get_matches",
        "description": "Return the rows in a DataFrame about women's football games which satisfy the criteria",
        "parameters": {
            "type": "object",
            "properties": {
                "country": {
                    "type": "string",
                    "description": "The name of the country the matches took place e.g. France or China",
                },
                "start_year": {
                    "type": "number",
                    "description": "The year to begin filtering from e.g. 1956",
                },
                "end_year": {
                    "type": "number",
                    "description": "The year to end filtering on e.g. 2005"}
            },
            "required": ["location", "start_year", "end_year"],
        },
    }
],
function_call="auto",
)

Converting the response to something we can pass into a locally defined function.

In [14]:
args = json.loads(completion.choices[0].message.function_call.arguments)
args

{'country': 'Italy', 'start_year': 1980, 'end_year': 1999}

#### Using arguments from our OpenAI Function call to interact with our locally defined function/ DataFrame

In [15]:
matches_finder(**args)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
114,1982-11-14,Italy,Portugal,3,0,UEFA Euro qualification,Genoa,Italy,False
140,1983-04-24,Italy,France,3,0,UEFA Euro qualification,Vicenza,Italy,False
155,1983-09-17,Italy,Switzerland,2,0,UEFA Euro qualification,Rome,Italy,False
171,1984-04-08,Italy,Sweden,2,3,UEFA Euro,Rome,Italy,False
176,1984-08-19,Italy,West Germany,1,2,Mundialito,Caorle,Italy,False
...,...,...,...,...,...,...,...,...,...
800,1995-10-21,Italy,Croatia,7,0,UEFA Euro qualification,Verona,Italy,False
835,1996-03-16,Italy,England,2,1,UEFA Euro qualification,Cosenza,Italy,False
845,1996-04-07,Italy,Portugal,4,1,UEFA Euro qualification,Mestre,Italy,False
1157,1999-10-13,Italy,Ukraine,1,0,UEFA Euro qualification,Castelfranco di Sotto,Italy,False


## RAG

<img src="https://github.com/toelt-llc/HSLU-NLP-Bootcamp/blob/main/GenAI/pics/lchain.png?raw=1" width="600"/>

### Working with embeddings and larger documents

In [16]:
# Creating embeddings
model = "text-embedding-ada-002"

embedding = openai.embeddings.create(input=["""This is a simple embedding of a sentence"""],
                                     model=model)

# How large are the embeddings we got?

import numpy as np

# Access the embedding data using the 'embeddings' attribute
embedding_data = embedding.data[0].embedding

# Convert the embedding data to a NumPy array
embedding_array = np.array(embedding_data)

# Get the shape of the embedding array
embedding_shape = embedding_array.shape

# Print the shape
print(embedding_shape)

(1536,)


Here, we download a book in PDF form that we can then use Langchain's document loader to prepare it for embedding

In [17]:
! wget -O book.pdf "https://greenteapress.com/thinkpython2/thinkpython2.pdf"

--2025-03-27 13:19:06--  https://greenteapress.com/thinkpython2/thinkpython2.pdf
Resolving greenteapress.com (greenteapress.com)... 67.205.24.128
Connecting to greenteapress.com (greenteapress.com)|67.205.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921415 (900K) [application/pdf]
Saving to: ‘book.pdf’

book.pdf            100%[===================>] 899.82K   860KB/s    in 1.0s    

2025-03-27 13:19:08 (860 KB/s) - ‘book.pdf’ saved [921415/921415]



In [21]:
from langchain.document_loaders import PyPDFLoader # Changed import statement

loader = PyPDFLoader("book.pdf")

data = loader.load()

To work with a large document, we need to split it into smaller chunks with one of Langchain's `text_splitter`s

In [22]:
import numpy as np

print (f'You have {len(data)} documents in your data')
print (f'''There are ~{np.mean([len(x.page_content) for x in data])} characters per document''')

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=400)

texts = text_splitter.split_documents(data)

You have 244 documents in your data
There are ~1820.1311475409836 characters per document


Next, we embed our documents directly into an in-memory vector database:

In [25]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

vector_db = Chroma.from_documents(texts,
                                  OpenAIEmbeddings(openai_api_key = openai_api_key,
                                                   model="text-embedding-ada-002"))

<img src="https://github.com/toelt-llc/HSLU-NLP-Bootcamp/blob/main/GenAI/pics/biencoder-diagram.png?raw=1" width="600"/>


We can then embed a sentence (e.g. a question) and see which of our texts are most similar to it.

In [26]:
# Querying the data
query = "How do I establish a Class?"
num_closest_docs = 5
docs = vector_db.similarity_search(query, k = num_closest_docs)
for k in range(num_closest_docs):
    print(f"""\n ~~~~~ Showing document #{k+1} ~~~~~ \n""")
    print(docs[k].page_content)


 ~~~~~ Showing document #1 ~~~~~ 

148 Chapter 15. Classes and objects
x
y
3.0
4.0
blank
Point
Figure 15.1: Object diagram.
The header indicates that the new class is called Point. The body is a docstring that ex-
plains what the class is for. You can deﬁne variables and methods inside a class deﬁnition,
but we will get back to that later.
Deﬁning a class named Point creates a class object.
>>> Point
<class '__main__.Point'>
Because Point is deﬁned at the top level, its “full name” is __main__.Point.
The class object is like a factory for creating objects. To create a Point, you callPoint as if it
were a function.
>>> blank = Point()
>>> blank
The return value is a reference to a Point object, which we assign to blank.
Creating a new object is called instantiation, and the object is an instance of the class.
When you print an instance, Python tells you what class it belongs to and where it is stored
in memory (the preﬁx 0x means that the following number is in hexadecimal).
Every obje

If we want, we can go further, passing this retrieved text as context for a prompt which we can then do question-answering on. Using `verbose = True` will allow you to see the chain of events taking place under the hood.

With Langchain, these pre-defined prompts can be altered for whatever purpose necessary.

In [27]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

llm = OpenAI(temperature=0,
             openai_api_key=openai_api_key,
             model = "gpt-3.5-turbo-instruct")

chain = load_qa_chain(llm,
                      chain_type="map_reduce",
                     verbose = True)

/tmp/ipykernel_36609/1849260753.py:4: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0,
/tmp/ipykernel_36609/1849260753.py:8: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.lan

**chain_type="map_reduce"** is one way to handle long or complex text when you want the LLM to analyze or summarize it.

🧩 The two steps:
1. Map Step (like chopping veggies)

- The document is split into smaller chunks.

- ChatGPT (or the LLM) reads each chunk separately and writes a small summary of each.

2. Reduce Step (like mixing ingredients)

- All those smaller summaries are then combined into one final summary.

- ChatGPT reads the summaries and creates a complete, clean result.

✅ Why use map_reduce?
- It helps when your document is too long for the model to read all at once.

- It makes processing more efficient and accurate.

- It lets the LLM stay within its token limits (i.e., memory).

🔎 A note on [temperature](https://blog.lukesalamone.com/posts/what-is-temperature/) and on ["map_reduce"](https://github.com/hwchase17/langchain-hub/blob/master/chains/question_answering/map-reduce/chain.json)!

In [28]:
query = "How do I define a class in Python"

docs = vector_db.similarity_search(query,
                                   k=5)
docs

[Document(metadata={'page': 168, 'source': 'book.pdf'}, page_content='Chapter 15\nClasses and objects\nAt this point you know how to use functions to organize code and built-in types to organize\ndata. The next step is to learn “object-oriented programming”, which uses programmer-\ndeﬁned types to organize both code and data. Object-oriented programming is a big topic;\nit will take a few chapters to get there.\nCode examples from this chapter are available from https://thinkpython.com/code/\nPoint1.py; solutions to the exercises are available fromhttps://thinkpython.com/code/\nPoint1_soln.py.\n15.1 Programmer-deﬁned types\nWe have used many of Python’s built-in types; now we are going to deﬁne a new type. As\nan example, we will create a type called Point that represents a point in two-dimensional\nspace.\nIn mathematical notation, points are often written in parentheses with a comma separating\nthe coordinates. For example, (0, 0) represents the origin, and (x, y) represents the poin

In [29]:
chain.run(input_documents=docs, question=query)

/tmp/ipykernel_36609/2708201610.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(input_documents=docs, question=query)




> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following portion of a long document to see if any of the text is relevant to answer the question. 
Return any relevant text verbatim.
Chapter 15
Classes and objects
At this point you know how to use functions to organize code and built-in types to organize
data. The next step is to learn “object-oriented programming”, which uses programmer-
deﬁned types to organize both code and data. Object-oriented programming is a big topic;
it will take a few chapters to get there.
Code examples from this chapter are available from https://thinkpython.com/code/
Point1.py; solutions to the exercises are available fromhttps://thinkpython.com/code/
Point1_soln.py.
15.1 Programmer-deﬁned types
We have used many of Python’s built-in types; now we are going to deﬁne a new type. As
an example, we will create a type called Point that represents a point in two-dimensional
space.
In mathemat

' To define a class in Python, use the keyword "class" followed by the name of the class. Within the class, define methods using the "def" keyword. Use the "__init__" method to initialize attributes and the "__str__" method to return a string representation of the object.'

## Further Readings

- [OpenAI API Docs](https://platform.openai.com/docs/concepts): Filled with code examples to use
- [Andrew Ng's Prompt Engineering](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/) for Developers: Excellent, free 1-hour course
